In [4]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyGraph").setMaster("local"))

In [5]:
raw_data = sc.textFile("/data/twitter/twitter_sample_small.txt")

In [7]:
! head -3 /data/twitter/twitter_sample_small.txt

12	2241
12	13349
12	41873


In [8]:
def parse_edge(s):
    user, follower = s.split("\t")
    return (int(user), int(follower))

In [9]:
edges = raw_data.map(parse_edge).cache()

In [10]:
edges.take(3)

[(12, 2241), (12, 13349), (12, 41873)]

In [19]:
import operator
follower_counts = edges.mapValues(lambda x: 1).reduceByKey(lambda a,b: a+b)

In [46]:
follower_counts.top(5, operator.itemgetter(1))

[(20, 12352), (13, 10333), (12, 10030), (586, 9261), (291, 9054)]

In [43]:
r = operator.itemgetter(1)

In [44]:
a=[1,2,3,4]
r(a)

2

https://docs.python.org/2.7/library/operator.html

 operator.itemgetter(item)
operator.itemgetter(*items)

    Return a callable object that fetches item from its operand using the operand’s __getitem__() method. If multiple items are specified, returns a tuple of lookup values. For example:

        After f = itemgetter(2), the call f(r) returns r[2].
        After g = itemgetter(2, 5, 3), the call g(r) returns (r[2], r[5], r[3]).

    Equivalent to:

    def itemgetter(*items):
        if len(items) == 1:
            item = items[0]
            def g(obj):
                return obj[item]
        else:
            def g(obj):
                return tuple(obj[item] for item in items)

In [47]:
follower_counts2 = edges.aggregateByKey(0, lambda a, x: a+1, operator.iadd)
follower_counts2.top(5,operator.itemgetter(1))

[(20, 12352), (13, 10333), (12, 10030), (586, 9261), (291, 9054)]

__aggregateByKey(zeroValue)(seqOp, combOp, [numPartitions])__ When called on a dataset of (K, V) pairs, returns a dataset of (K, U) pairs where the values for each key are aggregated using the given combine functions and a neutral "zero" value. Allows an aggregated value type that is different than the input value type, while avoiding unnecessary allocations. Like in groupByKey, the number of reduce tasks is configurable through an optional second argument. 